# Load libraries and data

In [ ]:
library(tidyverse)
library(phyloseq)
library(microbiome)
library(rstatix)
library(vegan)
library(ggrepel)
library(gridExtra)
library(rlang)
library(viridis)
library(ggpubr)
library(RVAideMemoire)

options(repr.plot.width=20, repr.plot.height=10)

In [ ]:
nasal_phylo_rarefied <- readRDS("../../results/Microbiome_analysis/nasal_samples_clean_rarefied.rds")
nasal_phylo_rarefied

In [ ]:
gut_phylo_rarefied <- readRDS("../../results/Microbiome_analysis/gut_samples_clean_rarefied.rds")
gut_phylo_rarefied

# Alpha diversity

## Functions

In [ ]:
# Calculate diversity index
calculate_diversity_index <- function(phyloseq, index) {
  estimate_richness(phyloseq, measures = c(index))[, index]
}

# Prepare data for plotting
prepare_plot_data <- function(phyloseq, comparisons, index) {
  # Extract index and timepoint data
  plot_data <- data.frame(
    diversity = calculate_diversity_index(phyloseq, index),
    timepoint = sample_data(phyloseq)$timepoint
  )
  
  # Convert timepoint to character and filter based on comparisons
  plot_data$timepoint <- as.character(plot_data$timepoint)
  timepoints_to_include <- unique(unlist(comparisons))
  plot_data <- plot_data[plot_data$timepoint %in% timepoints_to_include,]
  
  # Set the order of timepoints
  timepoint_order <- c("healthy day 0", "0", "28", "90", "180")
  plot_data$timepoint <- factor(plot_data$timepoint, levels = timepoint_order)
  
  return(plot_data)
}

# Perform statistical tests
perform_statistical_tests <- function(plot_data, comparisons, paired) {
  # Apply Wilcoxon test to each comparison
  stat_test <- do.call(rbind, lapply(comparisons, function(comp) {
    test_result <- wilcox.test(
			diversity ~ timepoint, 
			data = plot_data[plot_data$timepoint %in% comp,], 
			paired = paired
		)
    data.frame(
      .y. = "diversity",
      group1 = comp[1],
      group2 = comp[2],
      p = test_result$p.value,
      method = "Wilcoxon"
    )
  }))
  
  # Adjust p-values and add significance levels
  stat_test$p.adj <- p.adjust(stat_test$p, method = "BH")
  stat_test$p.adj.signif <- ifelse(
		stat_test$p.adj < 0.001, "***",
		ifelse(
			stat_test$p.adj < 0.01, "**",
				ifelse(stat_test$p.adj < 0.05, "*", "ns")
		)
	)
  
  return(stat_test)
}

# Create the plot
create_diversity_plot <- function(plot_data, stat_test, index) {

  options(repr.plot.width=20, repr.plot.height=10)
  # Calculate y-positions for p-value annotations
  max_value <- max(plot_data$diversity)
  y_positions <- seq(1.05, 1.05 + (0.05 * (nrow(stat_test) - 1)), by = 0.05) * max_value

  # Create color palette
  n_timepoints <- length(unique(plot_data$timepoint))
  color_palette <- viridis::viridis(n_timepoints)

  # Filter stat_test to only include significant results
  sig_stat_test <- stat_test[stat_test$p.adj < 0.05,]

  # Recalculate y-positions for only significant comparisons
  if(nrow(sig_stat_test) > 0) {
    y_positions <- seq(1.05, 1.05 + (0.05 * (nrow(sig_stat_test) - 1)), by = 0.05) * max_value
  }

  # Create the plot
  p <- ggplot(plot_data, aes(x = timepoint, y = diversity)) +
    geom_boxplot(aes(fill = timepoint), alpha = 0.7, outlier.shape = NA, color = "black") +    
    geom_jitter(aes(color = timepoint), width = 0.2, alpha = 0.5) +
    scale_fill_viridis(discrete = TRUE, option = "H") +
    scale_color_viridis(discrete = TRUE, option = "H") +
    theme_classic() +
    labs(x = "Timepoint", y = paste(index, "Diversity Index")) +
    theme(legend.position = "none",
          axis.text = element_text(size = 20),    
          axis.title = element_text(size = 25),
          plot.title = element_text(margin = margin(b = 50)))

  # Only add significance lines if there are significant results
  if(nrow(sig_stat_test) > 0) {
    p <- p + stat_pvalue_manual(
			sig_stat_test, 
			y.position = y_positions,
			step.increase = 0.05,
			label = "p.adj.signif"
		)
  }

  return(p)
}

# Main function to plot diversity
plot_diversity <- function(phyloseq, comparisons, paired, index, title) {
  # Prepare data
  plot_data <- prepare_plot_data(phyloseq, comparisons, index)
  
  # Perform statistical tests
  stat_test <- perform_statistical_tests(plot_data, comparisons, paired)
  
  # Create the plot
  p <- create_diversity_plot(plot_data, stat_test, index)
  
  # Add overall title with adjusted position
  titled_plot <- cowplot::ggdraw(ylim = c(0, 1.05)) + 
    cowplot::draw_plot(p, y = 0, height = 0.95) + 
    cowplot::draw_label(title, fontface = "bold", size = 24, x = 0.5, y = 1)
  
  # Display the plot
  print(titled_plot)
  
  # Print statistical test results
  cat("Diversity Statistical Test:\n")
  print(stat_test)
  
  return(titled_plot)
}

## All four timepoints

### Nasal passage microbiota

In [ ]:
comparisons1 <- list(
	c("0", "28"),
	c("0", "90"),
	c("0", "180"),
	c("28", "90"),
	c("28", "180"),
	c("90", "180")
)

In [ ]:
# subset phyloseq 
nasal_phylo_dupilumab <- subset_samples(nasal_phylo_rarefied, treatment == "Dupilumab_treatment")

#transform phyloseq into a df
nasal_phylo_dupilumab_df <- psmelt(nasal_phylo_dupilumab)

# Find the samples that are present at all timepoints
nasal_samples_at_all_timepoints <- nasal_phylo_dupilumab_df %>%
  group_by(patient) %>%
  filter(length(unique(timepoint)) == 4) %>%
  .$patient %>%
  unique()

# subset phyloseq to keep only those patients measured in all four timepoints
nasal_phylo_dupilumab_filtered_all <- subset_samples(nasal_phylo_dupilumab, patient %in% nasal_samples_at_all_timepoints)

In [ ]:
nasal_shannon_plot <- plot_diversity(
	nasal_phylo_dupilumab_filtered_all, 
	comparisons = comparisons1,
	paired = TRUE,
	index = "Shannon",
	title = "Nasal passage microbiota (Shannon)"
)

In [ ]:
# For Chao1 diversity
nasal_chao1_plot <- plot_diversity(
	nasal_phylo_dupilumab_filtered_all, 
	comparisons = comparisons1,
	paired = TRUE,
	index = "Chao1",
	title = "Nasal passage microbiota (Chao1)"
)

### Gut microbiota

In [ ]:
# subset phyloseq 
gut_phylo_dupilumab <- subset_samples(gut_phylo_rarefied, treatment == "Dupilumab_treatment")

#transform phyloseq into a df
gut_phylo_dupilumab_df <- psmelt(gut_phylo_dupilumab)

# Find the samples that are present at all timepoints
gut_samples_at_all_timepoints <- gut_phylo_dupilumab_df %>%
  group_by(patient) %>%
  filter(length(unique(timepoint)) == 4) %>%
  .$patient %>%
  unique()

# subset phyloseq to keep only those patients measured in all four timepoints
gut_phylo_dupilumab_filtered_all <- subset_samples(gut_phylo_dupilumab, patient %in% gut_samples_at_all_timepoints)

In [ ]:
gut_shannon_plot <- plot_diversity(
	gut_phylo_dupilumab_filtered_all, 
	comparisons = comparisons1,
	paired = TRUE,
	index = "Shannon",
	title = "Gut microbiota (Shannon)"
)

In [ ]:
# For Chao1 diversity
gut_chao1_plot <- plot_diversity(
	gut_phylo_dupilumab_filtered_all, 
	comparisons = comparisons1,
	paired = TRUE,
	index = "Chao1",
	title = "Gut microbiota (Chao1)"
)

### Day 0 healthy control vs Dupilumab treatment

In [ ]:
comparisons3 <- list(
    c("healthy day 0", "0"), 
    c("healthy day 0", "28"), 
    c("healthy day 0", "90"),
    c("healthy day 0", "180")
)

#### Nasal passage samples

In [ ]:
# subset phyloseq object to keep only dupilumab treated samples and healthy day 0
nasal_phylo_dupilumab <- subset_samples(nasal_phylo_rarefied, treatment == "Dupilumab_treatment")
nasal_phylo_healthy <- subset_samples(nasal_phylo_rarefied, treatment == "healthy_control" & timepoint == "0")

# Update sample data for healthy controls
sample_data(nasal_phylo_healthy)$timepoint <- "healthy day 0"

# Merge healthy controls with the existing dupilumab filtered data
nasal_phylo_modified <- merge_phyloseq(nasal_phylo_dupilumab, nasal_phylo_healthy)

# Update sample data for the merged phyloseq object
sample_data(nasal_phylo_modified)$timepoint <- factor(
  sample_data(nasal_phylo_modified)$timepoint,
  levels = c("healthy day 0", "0", "28", "90", "180")
)

In [ ]:
nasal_healthy_shannon_plot <- plot_diversity(
	nasal_phylo_modified, 
	comparisons = comparisons3,
	paired = FALSE,
	index = "Shannon",
	title = "Nasal passage microbiota (Shannon)"
)

In [ ]:
nasal_healthy_chao1_plot <- plot_diversity(
	nasal_phylo_modified, 
	comparisons = comparisons3,
	paired = FALSE,
	index = "Chao1",
	title = "Nasal passage microbiota (Chao1)"
)

#### Gut samples

In [ ]:
# subset phyloseq object to keep only dupilumab treated samples and healthy day 0
gut_phylo_dupilumab <- subset_samples(gut_phylo_rarefied, treatment == "Dupilumab_treatment")
gut_phylo_healthy <- subset_samples(gut_phylo_rarefied, treatment == "healthy_control" & timepoint == "0")

# Update sample data for healthy controls
sample_data(gut_phylo_healthy)$timepoint <- "healthy day 0"

# Merge healthy controls with the existing dupilumab filtered data
gut_phylo_modified <- merge_phyloseq(gut_phylo_dupilumab, gut_phylo_healthy)

# Update sample data for the merged phyloseq object
sample_data(gut_phylo_modified)$timepoint <- factor(
  sample_data(gut_phylo_modified)$timepoint,
  levels = c("healthy day 0", "0", "28", "90", "180")
)

In [ ]:
gut_healthy_shannon_plot <- plot_diversity(
	gut_phylo_modified, 
	comparisons = comparisons3,
	paired = FALSE,
	index = "Shannon",
	title = "Gut passage microbiota (Shannon)"
)

In [ ]:
gut_healthy_chao1_plot <- plot_diversity(
	gut_phylo_modified, 
	comparisons = comparisons3,
	paired = FALSE,
	index = "Chao1",
	title = "Gut passage microbiota (Chao1)"
)

# Beta diversity

## Functions

In [ ]:
perform_ordination <- function(phyloseq_object, group, distance_method = "bray", ordination_method = "PCoA", dimensions = 3, paired = FALSE, alpha = 0.05, seed = 123) {
  # Set random seed for reproducibility
  set.seed(seed)
  
  # Calculate the distance
  dist <- phyloseq::distance(phyloseq_object, method = distance_method)
  
  # Perform ordination
  ord <- ordinate(phyloseq_object, method = ordination_method, distance = distance_method, k = dimensions, trace = FALSE)
  
  # Extract the sample data from the phyloseq object
  sample_data <- sample_data(phyloseq_object)
  
  # Convert your grouping variable to a factor
  group_factor <- as.factor(sample_data[[group]])
  
  # Perform PERMANOVA based on paired argument
  if (paired) {
    # Set seed again before PERMANOVA
    set.seed(seed)
    # Perform pairwise PERMANOVA with BH correction
    permanova <- RVAideMemoire::pairwise.perm.manova(dist, group_factor, p.method = "BH")
  } else {
    # Set seed again before PERMANOVA
    set.seed(seed)
    # Perform unpaired PERMANOVA
    permanova <- vegan::adonis2(dist ~ group_factor, permutations = 999)
    
    # Check if the overall PERMANOVA is significant
    if (permanova$`Pr(>F)`[1] <= alpha) {
      # Perform pairwise comparisons
      levels <- levels(group_factor)
      n_levels <- length(levels)
      
      pairwise_results <- list()
      p_values <- c()
      
      for (i in 1:(n_levels - 1)) {
        for (j in (i + 1):n_levels) {
          subset_data <- phyloseq::prune_samples(sample_data[[group]] %in% c(levels[i], levels[j]), phyloseq_object)
          subset_dist <- phyloseq::distance(subset_data, method = distance_method)
          subset_group <- as.factor(sample_data(subset_data)[[group]])
          
          # Set seed again before each pairwise comparison
          set.seed(seed)
          pairwise_permanova <- vegan::adonis2(subset_dist ~ subset_group, permutations = 999)
          pairwise_results[[paste(levels[i], levels[j], sep = " vs ")]] <- pairwise_permanova
          p_values <- c(p_values, pairwise_permanova$`Pr(>F)`[1])
        }
      }
      
      # Adjust p-values using BH method
      adjusted_p_values <- p.adjust(p_values, method = "BH")
      
      # Add adjusted p-values to the pairwise results
      for (i in seq_along(pairwise_results)) {
        pairwise_results[[i]]$`Adjusted Pr(>F)` <- adjusted_p_values[i]
      }
      
      permanova$pairwise_results <- pairwise_results
    } else {
      message("Overall PERMANOVA not significant. Pairwise comparisons not performed.")
    }
  }
  
  # For pairwise results
  if (!is.null(permanova$pairwise_results)) {
    for (i in names(permanova$pairwise_results)) {
      cat("\nPairwise comparison:", i, "\n")
      print(permanova$pairwise_results[[i]], digits = 4, width = 1000)
    }
  } else {
    print(permanova)
  }

  # Return a named list
  return(list(ordination = ord, permanova = permanova))
}

In [ ]:
plot_nmds <- function(phyloseq_object, nmds_results, group, title, axes = c(1, 2)){
	plot <- plot_ordination(physeq = phyloseq_object, ordination = nmds_results[[1]], axes = axes, color = group) +
		geom_point(size = 4) +
		#ggrepel::geom_text_repel(aes(label = patient)) +
		ggtitle(title) +
		stat_ellipse() +
		scale_color_viridis_d(option = "H") + 
		theme_classic() +
		theme(
			axis.title = element_text(size = 24),  
			axis.text = element_text(size = 24),
			plot.title = element_text(size = 26)

		)
	
	return(plot)
}

In [ ]:
plot_multiple_nmds <- function(phyloseq_object, nmds_results, group, title){

	options(repr.plot.width=40, repr.plot.height=10)

	#create a function to extract the legend
	get_legend <- function(my_plot){
		tmp <- ggplot_gtable(ggplot_build(my_plot))
		leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box")
		legend <- tmp$grobs[[leg]]
		return(legend)
	}

	# Generate the three plots without legend
	plot1 <- plot_nmds(phyloseq_object, nmds_results, group, title, c(1, 2)) + theme(legend.position = "none")
	plot2 <- plot_nmds(phyloseq_object, nmds_results, group, title, c(2, 3)) + theme(legend.position = "none")
	plot3 <- plot_nmds(phyloseq_object, nmds_results, group, title, c(1, 3)) + theme(legend.position = "none")

	# Generate a separate legend
	legend <- get_legend(plot_nmds(phyloseq_object, nmds_results, group, title, c(1, 2)) + 
		theme(
			legend.title = element_text(size = 20),
			legend.text = element_text(size = 20),
			legend.key.size = unit(2, "cm")
		)
	)

	# Combine the plots into a single ggplot object
	combined_plot <- ggpubr::ggarrange(
		plot1,
		plot2,
		plot3,
		legend,
		ncol = 4, widths = c(1.5, 1.5, 1.5, 0.5)
	)

	# Add a common title

	return(combined_plot)
}


## Comaprison of healthy at day 0 to Dupilumab treatment

### Nasal passage samples

In [ ]:
# subset phyloseq object to keep only dupilumab treated samples and healthy day 0
nasal_phylo_dupilumab <- subset_samples(nasal_phylo_rarefied, treatment == "Dupilumab_treatment")
nasal_phylo_healthy <- subset_samples(nasal_phylo_rarefied, treatment == "healthy_control" & timepoint == "0")

# Update sample data for healthy controls
sample_data(nasal_phylo_healthy)$timepoint <- "healthy day 0"

# Merge healthy controls with the existing dupilumab filtered data
nasal_phylo_modified <- merge_phyloseq(nasal_phylo_dupilumab, nasal_phylo_healthy)

# Update sample data for the merged phyloseq object
sample_data(nasal_phylo_modified)$timepoint <- factor(
  sample_data(nasal_phylo_modified)$timepoint,
  levels = c("healthy day 0", "0", "28", "90", "180")
)

#### NMDS

In [ ]:
nasal_dupilumab_healthy_nmds <- perform_ordination(
	nasal_phylo_modified,
	"timepoint",
	"bray",
	"NMDS",
	dimensions = 3,
	paired = TRUE
)

nasal_dupilumab_healthy_nmds_plot <- plot_multiple_nmds(
	nasal_phylo_modified,
	nasal_dupilumab_healthy_nmds,
	"timepoint",
	"NMDS of nasal microbiota"
)

print(paste("Stress value: ", nasal_dupilumab_healthy_nmds[[1]]$stress))
nasal_dupilumab_healthy_nmds_plot

### Gut sampels

In [ ]:
# subset phyloseq object to keep only dupilumab treated samples and healthy day 0
gut_phylo_dupilumab <- subset_samples(gut_phylo_rarefied, treatment == "Dupilumab_treatment")
gut_phylo_healthy <- subset_samples(gut_phylo_rarefied, treatment == "healthy_control" & timepoint == "0")

# Update sample data for healthy controls
sample_data(gut_phylo_healthy)$timepoint <- "healthy day 0"

# Merge healthy controls with the existing dupilumab filtered data
gut_phylo_modified <- merge_phyloseq(gut_phylo_dupilumab, gut_phylo_healthy)

# Update sample data for the merged phyloseq object
sample_data(gut_phylo_modified)$timepoint <- factor(
  sample_data(gut_phylo_modified)$timepoint,
  levels = c("healthy day 0", "0", "28", "90", "180")
)

#### NMDS

In [ ]:
gut_dupilumab_healthy_nmds <- perform_ordination(
	gut_phylo_modified,
	"timepoint",
	"bray",
	"NMDS",
	dimensions = 3,
	paired = TRUE
)

gut_dupilumab_healthy_nmds_plot <- plot_multiple_nmds(
	gut_phylo_modified,
	gut_dupilumab_healthy_nmds,
	"timepoint",
	"NMDS of gut microbiota"
)

gut_dupilumab_healthy_nmds_plot
print(paste("Stress value: ", gut_dupilumab_healthy_nmds[[1]]$stress))

# Saving plots

In [ ]:
# Specify the path where you want to save the plots
save_path <- "../../results/Microbiome_analysis/plots/shannon"

# Create the directory if it doesn't exist
if (!dir.exists(save_path)) {
  dir.create(save_path, recursive = TRUE)
}

# Set plot dimensions and DPI
plot_width <- 10
plot_height <- 10
plot_dpi <- 300

# Get all variables ending with "_plot"
plot_vars <- ls(pattern = "shannon_plot$")

# Function to save a plot as both PNG and PDF
save_plots <- function(plot_name) {
  plot_obj <- get(plot_name)
  
  # Save as PNG
  png_filename <- file.path(save_path, paste0(plot_name, ".png"))
  pdf_filename <- file.path(save_path, paste0(plot_name, ".pdf"))
  
  # Check if it's a ggplot object
  if (inherits(plot_obj, "ggplot")) {
    ggsave(png_filename, plot = plot_obj, width = plot_width, height = plot_height, dpi = plot_dpi, units = "in")
    ggsave(pdf_filename, plot = plot_obj, width = plot_width, height = plot_height, units = "in")
  } else {
    # Assume it's a base R plot
    png(png_filename, width = plot_width, height = plot_height, units = "in", res = plot_dpi)
    print(plot_obj)
    dev.off()
    
    pdf(pdf_filename, width = plot_width, height = plot_height)
    print(plot_obj)
    dev.off()
  }
  
  cat("Saved:", png_filename, "\n")
  cat("Saved:", pdf_filename, "\n")
}

# Save each plot
invisible(sapply(plot_vars, save_plots))

cat("Saved", length(plot_vars), "plots (both PNG and PDF) in", save_path, "\n")
cat("Plot dimensions:", plot_width, "x", plot_height, "inches, DPI:", plot_dpi, "\n")

In [ ]:
# Specify the path where you want to save the plots
save_path <- "../../results/Microbiome_analysis/plots/chao1"

# Create the directory if it doesn't exist
if (!dir.exists(save_path)) {
  dir.create(save_path, recursive = TRUE)
}

# Set plot dimensions and DPI
plot_width <- 10
plot_height <- 10
plot_dpi <- 300

# Get all variables ending with "_plot"
plot_vars <- ls(pattern = "chao1_plot$")

# Function to save a plot as both PNG and PDF
save_plots <- function(plot_name) {
  plot_obj <- get(plot_name)
  
  # Save as PNG
  png_filename <- file.path(save_path, paste0(plot_name, ".png"))
  pdf_filename <- file.path(save_path, paste0(plot_name, ".pdf"))
  
  # Check if it's a ggplot object
  if (inherits(plot_obj, "ggplot")) {
    ggsave(png_filename, plot = plot_obj, width = plot_width, height = plot_height, dpi = plot_dpi, units = "in")
    ggsave(pdf_filename, plot = plot_obj, width = plot_width, height = plot_height, units = "in")
  } else {
    # Assume it's a base R plot
    png(png_filename, width = plot_width, height = plot_height, units = "in", res = plot_dpi)
    print(plot_obj)
    dev.off()
    
    pdf(pdf_filename, width = plot_width, height = plot_height)
    print(plot_obj)
    dev.off()
  }
  
  cat("Saved:", png_filename, "\n")
  cat("Saved:", pdf_filename, "\n")
}

# Save each plot
invisible(sapply(plot_vars, save_plots))

cat("Saved", length(plot_vars), "plots (both PNG and PDF) in", save_path, "\n")
cat("Plot dimensions:", plot_width, "x", plot_height, "inches, DPI:", plot_dpi, "\n")

In [ ]:
# Specify the path where you want to save the plots
save_path <- "../../results/Microbiome_analysis/plots/pcoa"

# Create the directory if it doesn't exist
if (!dir.exists(save_path)) {
  dir.create(save_path, recursive = TRUE)
}

# Set plot dimensions and DPI
plot_width <- 40
plot_height <- 10
plot_dpi <- 300

# Get all variables ending with "_plot"
plot_vars <- ls(pattern = "pcoa_plot$")

# Function to save a plot as both PNG and PDF
save_plots <- function(plot_name) {
  plot_obj <- get(plot_name)
  
  # Save as PNG
  png_filename <- file.path(save_path, paste0(plot_name, ".png"))
  pdf_filename <- file.path(save_path, paste0(plot_name, ".pdf"))
  
  # Check if it's a ggplot object
  if (inherits(plot_obj, "ggplot")) {
    ggsave(png_filename, plot = plot_obj, width = plot_width, height = plot_height, dpi = plot_dpi, units = "in")
    ggsave(pdf_filename, plot = plot_obj, width = plot_width, height = plot_height, units = "in")
  } else {
    # Assume it's a base R plot
    png(png_filename, width = plot_width, height = plot_height, units = "in", res = plot_dpi)
    print(plot_obj)
    dev.off()
    
    pdf(pdf_filename, width = plot_width, height = plot_height)
    print(plot_obj)
    dev.off()
  }
  
  cat("Saved:", png_filename, "\n")
  cat("Saved:", pdf_filename, "\n")
}

# Save each plot
invisible(sapply(plot_vars, save_plots))

cat("Saved", length(plot_vars), "plots (both PNG and PDF) in", save_path, "\n")
cat("Plot dimensions:", plot_width, "x", plot_height, "inches, DPI:", plot_dpi, "\n")

In [ ]:
# Specify the path where you want to save the plots
save_path <- "../../results/Microbiome_analysis/plots/nmds"

# Create the directory if it doesn't exist
if (!dir.exists(save_path)) {
  dir.create(save_path, recursive = TRUE)
}

# Set plot dimensions and DPI
plot_width <- 40
plot_height <- 10
plot_dpi <- 300

# Get all variables ending with "_plot"
plot_vars <- ls(pattern = "nmds_plot$")

# Function to save a plot as both PNG and PDF
save_plots <- function(plot_name) {
  plot_obj <- get(plot_name)
  
  # Save as PNG
  png_filename <- file.path(save_path, paste0(plot_name, ".png"))
  pdf_filename <- file.path(save_path, paste0(plot_name, ".pdf"))
  
  # Check if it's a ggplot object
  if (inherits(plot_obj, "ggplot")) {
    ggsave(png_filename, plot = plot_obj, width = plot_width, height = plot_height, dpi = plot_dpi, units = "in")
    ggsave(pdf_filename, plot = plot_obj, width = plot_width, height = plot_height, units = "in")
  } else {
    # Assume it's a base R plot
    png(png_filename, width = plot_width, height = plot_height, units = "in", res = plot_dpi)
    print(plot_obj)
    dev.off()
    
    pdf(pdf_filename, width = plot_width, height = plot_height)
    print(plot_obj)
    dev.off()
  }
  
  cat("Saved:", png_filename, "\n")
  cat("Saved:", pdf_filename, "\n")
}

# Save each plot
invisible(sapply(plot_vars, save_plots))

cat("Saved", length(plot_vars), "plots (both PNG and PDF) in", save_path, "\n")
cat("Plot dimensions:", plot_width, "x", plot_height, "inches, DPI:", plot_dpi, "\n")